#  tenserflow_mnist analysis

### MNIST数据集
位置：C:\Users\zjy\Desktop\jupyter notebook\MNIST_DATA

其中，一共四个文件，被分成两部分：60000行的训练数据集（mnist.train）和10000行的测试数据集（mnist.test）。

每一个MNIST数据单元有两部分组成：一张包含手写数字的图片和一个对应的标签。我们把这些图片设为“xs”，把这些标签设为“ys”。训练数据集和测试数据集都包含xs和ys，比如训练数据集的图片是 mnist.train.images ，训练数据集的标签是 mnist.train.labels。

在mnist模型中，每张图片包含28*28=784个像素点。因此，在MNIST训练数据集中，mnist.train.images 是一个形状为 [60000, 784] 的张量，第一个维度数字用来索引图片，第二个维度数字用来索引每张图片中的像素点。在此张量里的每一个元素，都表示某张图片里的某个像素的强度值，值介于0和1之间。

mnist.train.labels 是一个 [60000, 10] 的数字矩阵。标签数据是"one-hot vectors"。 一个one-hot向量除了某一位的数字是1以外其余各维度数字都是0。比如，标签0将表示成([1,0,0,0,0,0,0,0,0,0,0])。

### 解释模型：

X : 表示手写图片，[784]的张量
Y : 表示标签，[10]的张量

使用softmax回归模型。softmax模型可以用来给不同的对象分配概率。

第一步：获得evidence

对图片像素值进行加权求和。如果这个像素具有很强的证据说明这张图片不属于该类，那么相应的权值为负数，相反如果这个像素拥有有利的证据支持这张图片属于这个类，那么权值是正数。即：
$$evidence_{i} = \sum_{j}W_{i,j}+b_{i}  $$

其中，$W_{i}$代表权重，$b_{i}$代表偏置量bias

第二步：使用softmax函数可以把这些证据转换成概率 Y_predicted:
$$Y_predicted = softmax(evidence_{i})$$
这里的softmax把我们定义的线性函数的输出转换成我们想要的格式，也就是关于10个数字类的概率分布。因此，给定一张图片，它对于每一个数字的吻合度可以被softmax函数转换成为一个概率值。可以定义为：
$$softmax(evidence)= normalize(exp(evidence))$$
可以形象的表示为下图：
<img src="http://wiki.jikexueyuan.com/project/tensorflow-zh/images/softmax-regression-scalargraph.png" style="width:400px;height:200px">
改写成向量模式：
<img src="http://wiki.jikexueyuan.com/project/tensorflow-zh/images/softmax-regression-vectorequation.png" style="width:500px;height:200px">

或者可以写成以下方式：
$$Y_predicted = softmax(Wx+b)$$

In [1]:
""" 
tenserflow_mnist analysis
reference:
http://wiki.jikexueyuan.com/project/tensorflow-zh/tutorials/mnist_beginners.html
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import time

import utils

In [4]:
# Step 1: Read in data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/MNIST_DATA', one_hot = True)

Extracting /MNIST_DATA\train-images-idx3-ubyte.gz
Extracting /MNIST_DATA\train-labels-idx1-ubyte.gz
Extracting /MNIST_DATA\t10k-images-idx3-ubyte.gz
Extracting /MNIST_DATA\t10k-labels-idx1-ubyte.gz


In [5]:
#Step 2: 为features（X） and labels（Y）创建占位符

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None,10])

In [6]:
# Step 3: 创建w和b，以及模型
# W的维度是[784，10]，因为我们想要用784维的图片向量乘以它以得到一个10维的证据值向量，每一位对应不同数字类。

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

y_predicted = tf.nn.softmax(tf.matmul(x,W) + b)

至此，我们定义好了模型。为了训练我们的模型，我们首先需要定义一个指标来评估这个模型是好的。其实，在机器学习，我们通常定义指标来表示一个模型是坏的，这个指标称为成本（cost）或损失（loss），然后尽量最小化这个指标。但是，这两种方式是相同的。一个非常常见的，非常漂亮的成本函数是“交叉熵”（cross-entropy）。

#### “交叉熵”（cross-entropy）
定义如下：
$$ H_{y}(ypredicted)= -\sum_{i}y_{i}log(ypredicted_{i}) $$
在本模型中，可以用下面的公式计算交叉熵：
$$-\sum ylog(ypredicted)$$


In [7]:
cross_entropy = -tf.reduce_sum(y * tf.log(y_predicted))

我们要求TensorFlow用梯度下降算法（gradient descent algorithm）以0.01的学习速率最小化交叉熵。梯度下降算法（gradient descent algorithm）是一个简单的学习过程，TensorFlow只需将每个变量一点点地往使成本不断降低的方向移动。

TensorFlow在这里实际上所做的是，它会在后台给描述你的计算的那张图里面增加一系列新的计算操作单元用于实现反向传播算法和梯度下降算法。然后，它返回给你的只是一个单一的操作，当运行这个操作时，它用梯度下降算法训练你的模型，微调你的变量，不断减少成本。

In [8]:
learning_rate = 0.01
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

至此，我们定义好了tensorflow的graph。准备用一个session启动图。


In [12]:
#可以在一个Session里面启动我们的模型，并且初始化变量
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#然后开始训练模型，这里我们让模型循环训练1000次。
#该循环的每个步骤中，我们都会随机抓取训练数据中的100个批处理数据点，
#然后我们用这些数据点作为参数替换之前的占位符来运行train_step
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})

#### 评估模型

首先让我们找出那些预测正确的标签。

tf.argmax 是一个非常有用的函数，它能给出某个tensor对象在某一维上的其数据最大值所在的索引值。由于标签向量是由0,1组成，因此最大值1所在的索引位置就是类别标签，比如tf.argmax(y_predicted,1)返回的是模型对于任一输入x预测到的标签值，而 tf.argmax(y,1) 代表正确的标签，我们可以用 tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)。

In [14]:
# tf.equal()会给我们一组布尔值。为了确定正确预测项的比例，我们可以把布尔值转换成浮点数，然后取平均值。
#例如，[True, False, True, True] 会变成 [1,0,1,1] ，取平均值后得到 0.75.
correct_prediction = tf.equal(tf.argmax(y_predicted, 1), tf.argmax(y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

0.9025
